In [6]:
# Runtime ~1min 30sec -- 5000 tweets
# Runtime ~2min -- 10,000 tweets

In [7]:
award_names = ['Best Motion Picture – Drama', 'Best Motion Picture – Musical or Comedy', 'Best Motion Picture – Foreign Language',
 'Best Motion Picture – Animated', 'Best Director – Motion Picture', 'Best Actor in a Motion Picture – Drama',
 'Best Actor in a Motion Picture – Musical or Comedy', 'Best Actress in a Motion Picture – Drama',
 'Best Actress in a Motion Picture – Musical or Comedy', 'Best Supporting Actor – Motion Picture',
 'Best Supporting Actress – Motion Picture', 'Best Screenplay – Motion Picture', 'Best Original Score – Motion Picture',
 'Best Original Song – Motion Picture', 'Best Television Series – Drama', 'Best Television Series – Musical or Comedy',
 'Best Miniseries or Motion Picture – Television', 'Best Actor in a Television Series – Drama',
 'Best Actor in a Television Series – Musical or Comedy', 'Best Actor in a Miniseries or Motion Picture – Television',
 'Best Actress in a Television Series – Drama', 'Best Actress in a Television Series – Musical or Comedy',
 'Best Actress in a Miniseries or Motion Picture – Television',
 'Best Supporting Actor in a Series, Miniseries or Motion Picture – Television',
 'Best Supporting Actress in a Series, Miniseries or Motion Picture – Television']
new_award_names = [0]*len(award_names)
for i in range(len(award_names)):
    new_award_names[i] = award_names[i].lower()

In [8]:
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
from collections import defaultdict

In [9]:
df = pd.read_json('../gg2015.json')
df = df['text']
df = df[df.str.contains(' present', case = False)]
df = df[~df.str.contains('think|should|maybe', case = False)]
present_df = df.str.replace('#GoldenGlobes|golden|globes|globe', "", case = False)
if present_df.size > 10000:
    present_df = present_df.sample(10000)

In [10]:
from spacy.matcher import Matcher
from spacy.tokens import Span
nlp = spacy.load('en_core_web_md')
from string import punctuation

In [11]:
pattern1 = [{"ENT_TYPE": "PERSON", 'OP':'+'}, {"LEMMA": "present"}, {"ORTH":'Best'}, {"DEP": 'compound', 'OP':'+'}, {"POS":'NOUN', 'OP':'*'}, {'IS_PUNCT': True, 'OP':'?'}, {"POS": 'PROPN', 'OP':'*'}]
pattern2 = [{"ENT_TYPE": "WORK_OF_ART", 'OP':'+'}, {"LEMMA": "present"}, {"ORTH":'Best'}, {"DEP": 'compound', 'OP':'+'}, {"POS":'NOUN', 'OP':'*'}, {'IS_PUNCT': True, 'OP':'?'}, {"POS": 'PROPN', 'OP':'*'}]
pattern3 = [{"ENT_TYPE": "PERSON", 'OP':'+'}, {"LEMMA": "present"}, {"ORTH":'Best'}, {"ENT_TYPE": "WORK_OF_ART", 'OP':'+'}]
pattern4 = [{"ENT_TYPE": "WORK_OF_ART", 'OP':'+'}, {"LEMMA": "present"}, {"ORTH":'Best'}, {"ENT_TYPE": "WORK_OF_ART", 'OP':'+'}]

In [12]:
matcher = Matcher(nlp.vocab)
matcher.add("Presenter", [pattern1, pattern2, pattern3, pattern4])

In [13]:
presenters = {}
presenters = defaultdict(lambda: 1, presenters)
for i, text in present_df.iteritems():
    doc = nlp(text)
    matches = matcher(doc)
    if len(matches) != 0:
        span = doc[matches[-1][1]:matches[-1][2]]
        presenters[str(span).strip(punctuation).strip()] = presenters[str(span).strip(punctuation).strip()] +1

In [14]:
presenters = sorted(presenters.items(), key=lambda item: item[1], reverse = True)
print(presenters)

[('Robert Downey Jr. presents Best Movie (Comedy/Musical', 167), ('Prince presenting Best Original Song', 5), ('Bill Hader presented Best Screenplay', 4), ('Jennifer Aniston presenting Best Supporting Actor', 4), ('John Travolta presents Best Actress', 3), ('Seth Meyers presenting Best Supporting Actor', 3), ('Adam Levine present Best TV Series Drama', 3), ('Katherine Zeta-jones presents Best Film nominee #pride', 3), ('Meryl Streep presents Best Motion Picture', 2), ('JDornanLife: Video of Jamie and Dakota Presenting Best Supporting Actress Award http://t.co/bNYHgeofYg http://t.co/khuu8Wyn8Z” @stephee95', 2), ('Katie Holmes presented Best Supporting Actor', 2), ('Matthew Mcconaughey presents Best Actress for Motion Picture (Drama', 2), ('Kristen Wiig presenting Best Screenplay', 2), ('Jack Black presenting Best Film nominee', 2), ('Amy Adams presenting Best Actor Comedy Movie', 2), ('Naomi Watts presenting Best Picture nominee #Birdman http://t.co/h53gY3MOCX', 2), ('Meryl Streep prese

In [15]:
import Levenshtein
# tweet = string, awards = dict
def DistMeasure(tweet):
    awards = {}
    tweet = tweet.lower()
    tweet = tweet.replace('tv', 'television')
    tweet = tweet.replace('film', 'motion picture')
    tweet = tweet.replace('movie', 'motion picture')
    for award in new_award_names:
        split_award = award.split()
        award_dist = 0
        
        split_tweet = tweet.split()
        for word in split_tweet:
            array = []
            for word2 in split_award:
                array.append(Levenshtein.distance(word, word2))
            min_val = array[np.argmin(array)]
            award_dist += min_val
        awards[award] = award_dist
        
    val = min(awards.values())
    res = [key for key in awards if awards[key] == val]
    if len(res) > 10:
        res = []
    return res

In [16]:
award_list = [dict() for x in range(len(new_award_names))]

for tweet in presenters: 
    tweet = tweet[0]
    split = tweet.lower().split()
    
    index = split.index("best")

    award_phrase = " ".join(split[index:])
    matching_awards = DistMeasure(award_phrase)
    for a in matching_awards:
        award_index = new_award_names.index(a)  
        person = split[:index-1]
        person = " ".join(person)
        if person not in award_list[award_index]:
            award_list[award_index][person] = 1
        else:
            award_list[award_index][person] = award_list[award_index][person]+1

In [24]:
import operator

i = 0
awards = [[] for x in range(len(new_award_names))]
for award in award_list:
    if award != {}:
        max_val = max(award.items(), key=operator.itemgetter(1))[1]
        j = 0
        list_award = list(award)
        for b in award:
            if award[b] == max_val and len(awards[i]) < 2:
                awards[i].append(list_award[j])
            j+=1
    i+=1
print(awards)


[['channing tatum'], ['robert downey jr.'], [], [], ['harrison ford'], ['channing tatum'], ['robert downey jr.', 'amy adams'], ['channing tatum'], ['robert downey jr.', 'john travolta'], ['jennifer aniston'], ['john travolta', 'jared leto'], ['bill hader', 'kristen wiig'], ['vince vaughn'], ['prince'], ['adam levine'], [], ['melissa mccarthy', 'kate hudson'], ['adam levine'], [], ['melissa mccarthy', 'kate hudson'], ['john travolta', 'adam levine'], ['john travolta', 'anna faris'], ['john travolta', 'melissa mccarthy'], ['jennifer aniston'], ['john travolta', 'jdornanlife: video of jamie and dakota']]


In [18]:
result = dict(zip(award_names, awards)) 
print(result)

{'Best Motion Picture – Drama': ['channing tatum'], 'Best Motion Picture – Musical or Comedy': ['robert downey jr.'], 'Best Motion Picture – Foreign Language': [], 'Best Motion Picture – Animated': [], 'Best Director – Motion Picture': ['harrison ford'], 'Best Actor in a Motion Picture – Drama': ['channing tatum'], 'Best Actor in a Motion Picture – Musical or Comedy': ['robert downey jr.', 'amy adams'], 'Best Actress in a Motion Picture – Drama': ['channing tatum'], 'Best Actress in a Motion Picture – Musical or Comedy': ['robert downey jr.', 'john travolta'], 'Best Supporting Actor – Motion Picture': ['jennifer aniston'], 'Best Supporting Actress – Motion Picture': ['john travolta', 'jared leto'], 'Best Screenplay – Motion Picture': ['bill hader', 'kristen wiig'], 'Best Original Score – Motion Picture': ['vince vaughn'], 'Best Original Song – Motion Picture': ['prince'], 'Best Television Series – Drama': ['adam levine'], 'Best Television Series – Musical or Comedy': [], 'Best Miniseri